In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

# 模型
# 初始有100W$
#          总池：98W $  初始汇率：7
#        美元池：1W  $
#      人民币池：7W ￥
# 奇数期，卖，汇率涨（>=0.005）就将上一次买入美元池的美元卖到人民币池中
#            汇率降（>=0.005）就将上一次买入人民币池的人民币卖到总池中
# 偶数期，买，从总池中买1W美元到美元池，从总池中买1W人民币到人民币池中

In [2]:
rate = 7.0
nsteps = 100
draws = np.random.randint(0, 2, size=nsteps)
steps = np.where(draws>0, 0.995, 1.005)
steps

array([0.995, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005,
       0.995, 1.005, 0.995, 1.005, 1.005, 0.995, 0.995, 1.005, 1.005,
       1.005, 0.995, 1.005, 0.995, 0.995, 0.995, 1.005, 0.995, 1.005,
       1.005, 0.995, 1.005, 1.005, 1.005, 1.005, 0.995, 1.005, 1.005,
       0.995, 0.995, 1.005, 1.005, 1.005, 0.995, 1.005, 1.005, 1.005,
       1.005, 1.005, 0.995, 1.005, 0.995, 0.995, 0.995, 0.995, 1.005,
       1.005, 1.005, 0.995, 1.005, 1.005, 0.995, 0.995, 0.995, 1.005,
       1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 0.995, 1.005, 0.995,
       0.995, 0.995, 0.995, 1.005, 0.995, 0.995, 0.995, 1.005, 0.995,
       0.995, 0.995, 0.995, 1.005, 1.005, 0.995, 1.005, 1.005, 1.005,
       1.005, 0.995, 0.995, 1.005, 0.995, 0.995, 0.995, 1.005, 0.995,
       1.005])

In [3]:
truestep = steps.cumprod()
truestep

array([0.995     , 0.999975  , 1.00497487, 1.00999975, 1.01504975,
       1.020125  , 1.02522562, 1.03035175, 1.03550351, 1.03032599,
       1.03547762, 1.03030023, 1.03545173, 1.04062899, 1.03542585,
       1.03024872, 1.03539996, 1.04057696, 1.04577985, 1.04055095,
       1.0457537 , 1.04052493, 1.03532231, 1.0301457 , 1.03529643,
       1.03011994, 1.03527054, 1.0404469 , 1.03524466, 1.04042089,
       1.04562299, 1.0508511 , 1.05610536, 1.05082483, 1.05607896,
       1.06135935, 1.05605256, 1.05077229, 1.05602615, 1.06130629,
       1.06661282, 1.06127975, 1.06658615, 1.07191908, 1.07727868,
       1.08266507, 1.0880784 , 1.082638  , 1.08805119, 1.08261094,
       1.07719788, 1.07181189, 1.06645283, 1.0717851 , 1.07714402,
       1.08252974, 1.0771171 , 1.08250268, 1.08791519, 1.08247562,
       1.07706324, 1.07167792, 1.07703631, 1.0824215 , 1.0878336 ,
       1.09327277, 1.09873913, 1.10423283, 1.10975399, 1.10420522,
       1.10972625, 1.10417762, 1.09865673, 1.09316345, 1.08769

In [4]:
truerate = truestep * rate 
truerate

array([6.965     , 6.999825  , 7.03482412, 7.06999825, 7.10534824,
       7.14087498, 7.17657935, 7.21246225, 7.24852456, 7.21228194,
       7.24834335, 7.21210163, 7.24816214, 7.28440295, 7.24798094,
       7.21174103, 7.24779974, 7.28403873, 7.32045893, 7.28385663,
       7.32027592, 7.28367454, 7.24725616, 7.21101988, 7.24707498,
       7.21083961, 7.24689381, 7.28312828, 7.24671263, 7.2829462 ,
       7.31936093, 7.35595773, 7.39273752, 7.35577383, 7.3925527 ,
       7.42951547, 7.39236789, 7.35540605, 7.39218308, 7.429144  ,
       7.46628972, 7.42895827, 7.46610306, 7.50343357, 7.54095074,
       7.57865549, 7.61654877, 7.57846603, 7.61635836, 7.57827657,
       7.54038518, 7.50268326, 7.46516984, 7.50249569, 7.54000817,
       7.57770821, 7.53981967, 7.57751877, 7.61540636, 7.57732933,
       7.53944268, 7.50174547, 7.5392542 , 7.57695047, 7.61483522,
       7.6529094 , 7.69117394, 7.72962981, 7.76827796, 7.72943657,
       7.76808375, 7.72924334, 7.69059712, 7.65214413, 7.61388

In [5]:
percnt = 10000
rmb = 70000
doller = 10000
pool = 98 * 10000
pooltime = [100*10000]
for i in range(100):
    if(i%2 == 0): 
        rmb += percnt * truerate[i]
        doller += percnt
        pool -= (percnt*2)
    else:
        if(steps[i] == 0.995):
            ratelast = rate if(i==0) else truerate[i-1]
            rmb -= percnt * ratelast
            pool += percnt * ratelast / truerate[i]
        else:
            doller -= percnt
            rmb += percnt * truerate[i]
    # pooltime.append[rmb/truerate[i]+doller+pool]
    print(i, '\t', rmb, '\t', rmb/truerate[i], '\t', doller, '\t', pool, '\t', rmb/truerate[i]+doller+pool, '\t', truerate[i])

0 	 139650.0 	 20050.251256281408 	 20000 	 960000 	 1000050.2512562814 	 6.965
1 	 209648.25 	 29950.49876246907 	 10000 	 960000 	 999950.498762469 	 6.999824999999999
2 	 279996.49124999996 	 39801.491305939366 	 20000 	 940000 	 999801.4913059394 	 7.034824124999998
3 	 350696.47370624996 	 49603.47393625809 	 10000 	 940000 	 999603.4739362581 	 7.069998245624998
4 	 421749.9560747812 	 59356.6904838389 	 20000 	 920000 	 999356.6904838389 	 7.105348236853121
5 	 493158.7058551551 	 69061.38356600887 	 10000 	 920000 	 999061.3835660089 	 7.140874978037386
6 	 564924.4993844308 	 78717.79459304365 	 20000 	 900000 	 998717.7945930436 	 7.176579352927573
7 	 637049.121881353 	 88326.1637741728 	 10000 	 900000 	 998326.1637741728 	 7.2124622496922095
8 	 709534.3674907597 	 97886.73012355505 	 20000 	 880000 	 997886.730123555 	 7.24852456094067
9 	 637049.121881353 	 88328.3719834724 	 20000 	 890050.2512562814 	 998378.6232397538 	 7.212281938135966
10 	 709532.5553596194 	 97888